## Install the requirements

In [1]:
!pip install -q llama-index==0.12.12 openai==1.59.6 tiktoken==0.8.0 llama-index-readers-web==0.3.4 firecrawl-py==1.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.

### SET THE ENVIRONMENT VARIABLES

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"
FIRECRAWL_API_KEY = "<FIRECRAWL_API_KEY>"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY1')
# os.environ["FIRECRAWL_API_KEY"] = userdata.get('FIRECRAWL_API_KEY')

# FIRECRAWL_API_KEY = userdata.get('FIRECRAWL_API_KEY')

# SCRAPE WITH FIRECRAWL

## IMPORT THE FIRECRAWL WEBREADER

Firecrawl allows you to turn entire websites into LLM-ready markdown

Get the API key here
https://www.firecrawl.dev/app/api-keys

In [3]:
from llama_index.readers.web import FireCrawlWebReader

In [4]:

# using firecrawl to crawl a website
firecrawl_reader = FireCrawlWebReader(
    api_key=FIRECRAWL_API_KEY,  # Replace with your actual API key from https://www.firecrawl.dev/
    mode="scrape",
)

# Load documents from a single page URL
documents = firecrawl_reader.load_data(url="https://towardsai.net/")

In [5]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

Metadata length (977) is close to chunk size (1024). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.


In [6]:
res = query_engine.query("What is towards AI aim?")

print(res.response)

print("-----------------")
# Show the retrieved nodes
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("URL\t", src.metadata['sourceURL'])
  print("Score\t", src.score)
  print("Description\t", src.metadata.get("description"))
  print("-_"*20)

Towards AI aims to be an online publication that focuses on sharing high-quality publications, news, articles, and stories on AI and technology-related topics.
-----------------
Node ID	 3439640a-17d2-4518-8df7-f997ef3aecf9
Title	 Towards AI
URL	 https://towardsai.net/
Score	 0.884347259605429
Description	 ['Towards AI is an online publication, which focuses on sharing high-quality publications, news, articles, and stories on AI and technology related topics.', 'Our thoughts and highlights from the latest AI news, models and papers. Read by over 120,000 AI practitioners and students. Click to read Towards AI Newsletter, a Substack publication.']
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Node ID	 7cbaca1d-097f-4e8b-9bfc-551d5b38b209
Title	 Towards AI
URL	 https://towardsai.net/
Score	 0.8843338180104813
Description	 ['Towards AI is an online publication, which focuses on sharing high-quality publications, news, articles, and stories on AI and technology related topics.', 'Our thoughts an

# CRAWL A WEBSITE

## Load The CSV

CSV contains the list of tools and url of the page which we use to get information about the tool.

In [7]:
import requests
import csv

# Google Sheets file URL (CSV export link)
url = 'https://docs.google.com/spreadsheets/d/1gHB-aQJGt9Nl3cyOP2GorAkBI_Us2AqkYnfqrmejStc/export?format=csv'

# Send a GET request to fetch the CSV file
response = requests.get(url)

response_list = []
# Check if the request was successful
if response.status_code == 200:
    # Decode the content to a string
    content = response.content.decode('utf-8')

    # Use the csv.DictReader to read the content as a dictionary
    csv_reader = csv.DictReader(content.splitlines(), delimiter=',')
    response_list = [row for row in csv_reader]
else:
    print(f"Failed to retrieve the file: {response.status_code}")


In [8]:
import random

start_index = random.randint(0, len(response_list) - 3)
website_list = response_list[start_index:start_index+10] # crawling 2 website for demo

In [9]:
import pprint
print("CSV data")
pprint.pprint(website_list)

CSV data
[{'': '',
  'Category': 'Model Deployment',
  'Company': '',
  'Description': 'Deep learning deployment toolkit',
  'Is a direct URL company /tool website?': 'Yes',
  'Name': 'OpenVINO',
  'Parent': '',
  'Tool Type': 'Toolkit',
  'URL': 'https://docs.openvino.ai/2024/index.html'},
 {'': '',
  'Category': 'Vector Database',
  'Company': '',
  'Description': 'Vector database for machine learning applications',
  'Is a direct URL company /tool website?': 'Yes',
  'Name': 'Pinecone',
  'Parent': '',
  'Tool Type': 'Vector Database',
  'URL': 'https://www.pinecone.io/'},
 {'': '',
  'Category': 'Vector Database',
  'Company': '',
  'Description': 'Vector similarity search engine',
  'Is a direct URL company /tool website?': 'Yes',
  'Name': 'Qdrant',
  'Parent': '',
  'Tool Type': 'Vector Database',
  'URL': 'https://qdrant.tech/'},
 {'': '',
  'Category': 'Vector Database',
  'Company': '',
  'Description': 'Cloud-native, real-time vector database',
  'Is a direct URL company /to

## Initialize the Firecrawl

In [10]:
import os
from firecrawl import FirecrawlApp
app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)

In [11]:
import time

# Crawl websites and handle responses
url_response = {}
crawl_per_min = 1  # Max crawl per minute

# Track crawls
crawled_websites = 0
scraped_pages = 0

for i, website_dict in enumerate(website_list):
    url = website_dict.get('URL')
    print(f"Crawling: {url}")

    try:
        response = app.crawl_url(
            url,
            params={
                'limit': 5,  # Limit pages to scrape per site.
                'scrapeOptions': {'formats': ['markdown', 'html']}
            }
        )
        crawled_websites += 1

    except Exception as exc:
        print(f"Failed to fetch {url} -> {exc}")
        continue

    # Store the scraped data and associated info in the response dict
    url_response[url] = {
        "scraped_data": response.get("data"),
        "csv_data": website_dict
    }

    # Pause to comply with crawl per minute limit for free version its 1 crawl per minute
    if i!=len(website_list) and (i + 1) % crawl_per_min == 0:
        print("Pausing for 1 minute to comply with crawl limit...")
        time.sleep(60)  # Pause for 1 minute after every crawl


Crawling: https://docs.openvino.ai/2024/index.html
Pausing for 1 minute to comply with crawl limit...
Crawling: https://www.pinecone.io/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://qdrant.tech/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://weaviate.io/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://vespa.ai/
Pausing for 1 minute to comply with crawl limit...


## Create  llamaindex documents from the scraped content

In [12]:
from llama_index.core import Document
documents = []

for _, scraped_content in url_response.items():
    csv_data = scraped_content.get("csv_data")
    scraped_results = scraped_content.get("scraped_data")

    for scraped_site_dict in scraped_results:
        for result in scraped_results:
            markdown_content = result.get("markdown")
            title = result.get("metadata").get("title")
            url = result.get("metadata").get("sourceURL")
            documents.append(
                Document(
                    text=markdown_content,
                    metadata={
                        "title": title,
                        "url": url,
                        "description": csv_data.get("Description"),
                        "category": csv_data.get("Category")
                    }
                )
            )


# Create The RAG Pipeline.

In [13]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter

llm = OpenAI(model="gpt-4o-mini")
embed_model = OpenAIEmbedding(model="text-embedding-3-large")
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=30)

In [14]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

In [15]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [16]:
from IPython.display import Markdown, display
def display_response(response):
    display(Markdown(f"<b>{response}</b>"))

In [19]:
query = "What is Qdrant Database?" # Enter your query here, it should be relevant to the crawled websites
res = query_engine.query(query)
display_response(res)

print("-----------------")
# Show the retrieved nodes
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("URL\t", src.metadata['url'])
  print("Score\t", src.score)
  print("Description\t", src.metadata.get("description"))
  print("Category\t", src.metadata.get("category"))
  print("-_"*20)

<b>Qdrant Database is an open-source vector similarity search engine designed to handle high-dimensional vectors, making it suitable for performance-intensive and large-scale AI applications. It offers features such as cloud-native scalability, ease of deployment with Docker, cost-efficient storage options, and is built for reliability and speed using Rust.</b>

-----------------
Node ID	 48b9eb36-751d-4709-84ab-a294726da974
Title	 Qdrant - Vector Database - Qdrant
URL	 https://qdrant.tech/
Score	 0.5718558757273637
Description	 Vector similarity search engine
Category	 Vector Database
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
